<a href="https://colab.research.google.com/github/DhanieArdiyan/ASE2306-Capstone-Project/blob/main/Capstone_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install stopwords

     |████████████████████████████████| 51kB 2.4MB/s 
  Created wheel for stopwords: filename=stopwords-0.1.3-py2.py3-none-any.whl size=37286 sha256=56328952168d9219ea602afdc510279ea3c8089eff809edfec1953c28f949cab
  Stored in directory: /root/.cache/pip/wheels/39/fa/c7/c4c5111e658f5c58465d948165dc3395a3c10ff57f4cd20356
Successfully built stopwords


In [ ]:
pip install nltk

In [ ]:
#START INITIALISATION
from textblob import TextBlob
from wordcloud import WordCloud
import re
import pandas as pd 
import numpy as np 
import sys, tweepy
plt.style.use('fivethirtyeight')

def percentage(part, whole):
	return 100 * float(part)/float(whole)

consumer_key = "duCD34AKM0yj2FJGe1vOomirc"
consumer_secret = "aKkRF84oLs3pGm5gJzPavojoI23WZXMxrv4FWGZMKWnnSCoF1r"
access_token = "1284696550740901893-xwi5gsHUdnwnD1ztZbSyVdhe6ZWdev"
access_token_secret = "y8yNJJIxXtKiUgeY6kh8xWNSGEssNdCAdTIwQqnqR57yk"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth, wait_on_rate_limit=True) 

In [ ]:
#EXTRACTING TWEETS
keyWord = input("Enter the keyword you are searching for (skip if NA):  ") #search by keyword
UserName = input("Enter the Twitter username (excluding @, skip if NA): ") #search by username
inputString = keyWord +" from:" + UserName

public_tweets = api.search(q=inputString, count=1000, lang = "en")

i = 1
for tweet in public_tweets:
  print ('\n')
  print ((str(i) + ') ' + tweet.text))
  print (('Tweeted by') + ': ' + str(tweet.user.screen_name))
  print (('Tweeted at') + ': ' + str(tweet.user.location))
  print (('Tweeted on') + ': ' + str(tweet.created_at))
  i=i+1
  # print(tweet.text)

#IF I want to print the latest 5 tweets from the account
# print("Show the 5 recent tweets: \n")
# i = 1
# for tweet in posts [0:5]:
#     print ((str(i) + ') ' + tweet.full_text + '\n'))
#     i = i + 1

#Creating a dataframe with a column called Tweets
df = pd.DataFrame ([tweet.text for tweet in public_tweets], columns = ['Tweets'])

#Show first 5 rows of data
df.head()

#function to clean the tweets
def cleanTxt(text):
	text = re.sub(r'@[A-Za-z0-9]+', '', text) #removes @mentions
	text = re.sub(r'#', '', text) #removes the '#'
	text = re.sub(r'RT[\s+]+', '', text) #removing RT
	text = re.sub(r'https?:\/\/\S+', '', text) #removes hyperlinks
	return text

#cleaning the text
df['Tweets'] = df['Tweets'].apply(cleanTxt)

#Show the cleaned text
df

#Subjectivity function
def getSubjectivity(text):
	return TextBlob(text).sentiment.subjectivity

#Polarity function
def getPolarity(text):
	return TextBlob(text).sentiment.polarity

#Creating columns for Subjectivity and Polarity
df ['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df ['Polarity'] = df['Tweets'].apply(getPolarity)

#Show the new dataframe with the new columns
df


In [ ]:
# #Plotting WORDCLOUD
allWords = ' '.join([twts for twts in df['Tweets']])
wordcloud = WordCloud(width = 5000, height = 300, random_state = 21, max_font_size = 100).generate(allWords)

plt.imshow(WordCloud, interpolation = "bilinear")
plt.axis('off')
plt.show()



In [ ]:
#Computing the negative, neutral, positive analysis
def getAnalysis (score):
	if score < 0:
		return 'Negative'
	elif score == 0:
		return 'Neutral'
	else:
		return 'Positive'

df['Analysis'] = df['Polarity'].apply(getAnalysis)
df


In [ ]:
#To seperate and print POSITIVE tweets
j = 1
sortedDF = df.sort_values(by=['Polarity'])
for i in range (0, sortedDF.shape[0]):
	if (sortedDF['Analysis'][i] == 'Positive'):
		print (str(j) + ') '+sortedDF['Tweets'][i])
		print()
		j = j + 1


In [ ]:
#To seperate and print NEGATIVE tweets
j = 1
sortedDF = df.sort_values(by=['Polarity'], ascending='False')
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Negative'):
    print (str(j) + ') '+sortedDF['Tweets'][i])
    print()
    j = j + 1

In [ ]:
#POSITIVE TWEETS in %
ptweets = df[df.Analysis == 'Positive']
ptweets = ptweets['Tweets']

positive = round ( (ptweets.shape[0]/df.shape[0]) * 100, 1)
positive

In [ ]:
#NEGATIVE TWEETS in %
ntweets = df[df.Analysis == 'Negative']
ntweets = ntweets['Tweets']

negative = round ( (ntweets.shape[0]/df.shape[0]) * 100, 1)
negative

In [ ]:
#NEUTRAL TWEETS in %
100 - positive - negative

In [ ]:
#BAR GRAPH - POSITIVE, NEGATIVE, NEUTRAL
df['Analysis'].value_counts()

#plot and visualise the counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
#SCATTER PLOT - POLARITY AND SUBJECTIVITY
plt.figure(figsize = (8,6))
for i in range (0,df.shape[0]):
  plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color='Blue') #(x,y axis)

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
#ATTEMPT 1 TO MEASURE ACCURACY
#METHOD: USING LSTM & RNN
#link: https://analyticsindiamag.com/how-to-implement-lstm-rnn-network-for-sentiment-analysis/

In [ ]:
# from google.colab import files

# df.to_csv('insertcsvfilehere.csv')
# files.download('insertcsvfilehere.csv')

In [ ]:
# import numpy as np
# import pandas as pd
# from keras.models import Sequential
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.np_utils import to_categorical
# import re

# from google.colab import files

# df.to_csv('f1.csv')
# files.download('f1.csv')

# print(df.columns)

In [ ]:
# new_df = df[['text','sentiment']]

# print(data.sentiment)

In [ ]:
# new_df = new_df[new_df.sentiment != "Neutral"]
# new_df['text'] = new_df['text'].str.lower()
# new_df['text'] = new_df['text'].re.sub('[^a-zA-z0-9\s]')

# tokenizer = Tokenizer(num_words=1500, split=' ')
# tokenizer.fit_on_texts(data['text'].values)
# X = tokenizer.texts_to_sequences(new_df['text'])
# X = pad_sequences(X)

# embed_dim = 128
# lstm_out = 196
# model = Sequential()
# model.add(Embedding(vocabSize, embed_dim,input_length = 28)) 

# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(2,activation='softmax'))
# model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# Le = LabelEncoder()
# y = Le.fit_transform(new_df['sentiment'])

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, random_state = 42)
# model.fit(X_train, Y_train,validation_data = (X_test,y_test),epochs = 10, batch_size=32)

# model.evaluate(X_test,y_test)

In [ ]:
# print("Prediction: ",model.predict_classes(X_test[5:10]))
# print("Actual: \n",y_test[5:10])

In [ ]:
#ATTEMPT 2 TO MEASURE ACCURACY
#METHOD: Training model by RandomForestClassifier algorithm
#link: https://stackabuse.com/python-for-nlp-sentiment-analysis-with-scikit-learn/

In [ ]:
# import matplotlib.pyplot as plt 
# import nltk
# from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# processed_features = []
# features = df['Tweets']


# for sentence in range(0, len(features)):
#     # Remove all the special characters
#     processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

     # remove all single characters
#     processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

#     # Remove single characters from the start
#     processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

#     # Substituting multiple spaces with single space
#     processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

#     # Removing prefixed 'b'
#     processed_feature = re.sub(r'^b\s+', '', processed_feature)

#     # Converting to Lowercase
#     processed_feature = processed_feature.lower()

#     processed_features.append(processed_feature)

# processed_features

In [ ]:
# from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
# processed_features = vectorizer.fit_transform(processed_features).toarray()

# #max_features = most frequently occuring words to create a bag of words feature vector
# #max_df = specifies to only using words that occur in a maximum of 80% of the docs
# #min_df = similar, include words that occur in at least 7 docs

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(processed_features, df, test_size=0.2, random_state=0)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import StandardScaler

# text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
# text_classifier.fit(X_train, y_train)

In [ ]:
# predictions = text_classifier.predict(X_test)

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# print(confusion_matrix(y_test,predictions))
# print(classification_report(y_test,predictions))
# print(accuracy_score(y_test, predictions))

In [ ]:
#ATTEMPT 3 TO MEASURE ACCURACY
#METHOD: Implementing SVM with Scikit-Learn
#link: https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

# bankdata = pd.read_csv("bill_authentication.csv")
# bankdata.shape
# bankdata.head()

# X = bankdata.drop('Class', axis=1)
# y = bankdata['Class']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# from sklearn.svm import SVC
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(X_train, y_train)

# y_pred = svclassifier.predict(X_test)

# from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))